# Talking Data

In [54]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import os
import sys
import time
import random
import tarfile
import zipfile
import StringIO
from IPython.display import display, Image

from scipy import ndimage
from sklearn.cross_validation import train_test_split, StratifiedShuffleSplit
from sklearn.linear_model import LogisticRegression
from sklearn.cluster import KMeans
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn import linear_model, decomposition, datasets, ensemble
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer,precision_score, recall_score, f1_score, average_precision_score, accuracy_score

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
from subprocess import check_output
#print(check_output(["ls", "../input"]).decode("utf8"))
# Any results you write to the current directory are saved as output.

datadir=".\data\TalkingData\\"
# Config the matlotlib backend as plotting inline in IPython
%matplotlib inline

### some function def

In [92]:
def loadData(datadir,filename):
    # Load the wholesale customers dataset
    #data = pd.read_csv(filename)
    data = ''
    print ("loading: "+datadir+filename)
    try:
        if zipfile.is_zipfile(datadir+filename):
            z = zipfile.ZipFile(datadir+filename)
            filename = z.open(filename[:-4])
        else:
            filename=datadir+filename
        data = pd.read_csv(filename, parse_dates=True)  
        print ("Dataset has {} samples with {} features each.".format(*data.shape))
    except Exception as e:
        print ("Dataset could not be loaded. Is the dataset missing?")
        print(e)
    return data

def writeData(data,filename):
    # Load the wholesale customers dataset
    try:
        data.to_csv(filename, index=False)
    except Exception as e:
        print ("Dataset could not be written.")
        print(e)
    verify=[]
    try:
        with open(filename, 'r') as f:
            for line in f:
                verify.append(line)
        f.closed
        return verify[:5]
    except IOError:
        sys.std

## Import data

In [106]:
app_labels = loadData(datadir,"app_labels.csv.zip")
#app_events = loadData(datadir,"app_events.csv.zip")
#events = loadData(datadir,"events.csv.zip")
gender_age_test = loadData(datadir,"gender_age_test.csv.zip") 
gender_age_train = loadData(datadir,"gender_age_train.csv.zip") 
label_categories = loadData(datadir,"label_categories.csv.zip") 
phone_brand_device_model = loadData(datadir,"phone_brand_device_model.csv.zip") 
brands = loadData(datadir,"brands.csv")

loading: .\data\TalkingData\app_labels.csv.zip
Dataset has 459943 samples with 2 features each.
loading: .\data\TalkingData\gender_age_test.csv.zip
Dataset has 112071 samples with 1 features each.
loading: .\data\TalkingData\gender_age_train.csv.zip
Dataset has 74645 samples with 4 features each.
loading: .\data\TalkingData\label_categories.csv.zip
Dataset has 930 samples with 2 features each.
loading: .\data\TalkingData\phone_brand_device_model.csv.zip
Dataset has 187245 samples with 3 features each.
loading: .\data\TalkingData\brands.csv
Dataset has 98 samples with 2 features each.


In [120]:
display (gender_age_train.head(3))

display(pd.merge(app_labels, label_categories, on='label_id', how='outer').head(3))

phone_brand_device_model_english=pd.merge(phone_brand_device_model, brands, on='phone_brand', how='outer')
phone_brand_device_model_english.drop("phone_brand", axis=1, inplace=True)

display(phone_brand_device_model_english.head(3))

gender_age_phone_brand_device=pd.merge(gender_age_train, phone_brand_device_model_english, on='device_id', how='inner')
display(gender_age_phone_brand_device.head(3))


,device_id,gender,age,group
0,-8076087639492063270,M,35,M32-38
1,-2897161552818060146,M,35,M32-38
2,-8260683887967679142,M,35,M32-38


,app_id,label_id,category
0,7.324885e+18,251,Finance
1,-4.494217e+18,251,Finance
2,8.756706e+18,251,Finance


,device_id,device_model,phone_brand_English
0,-8890648629457979026,红米,Xiaomi
1,1277779817574759137,MI 2,Xiaomi
2,-3883532755183027260,MI 2S,Xiaomi


,device_id,gender,age,group,device_model,phone_brand_English
0,-8076087639492063270,M,35,M32-38,MI 2,Xiaomi
1,-2897161552818060146,M,35,M32-38,MI 2,Xiaomi
2,-8260683887967679142,M,35,M32-38,MI 2,Xiaomi
